In [120]:
import requests
from bs4 import BeautifulSoup
import copy
import yaml
import os
import pandas as pd

path = "./data/ps_scrappe_babis2013.yaml"
key_word = "Andrej Babiš:"

important_words = ["předsed", "poslan", "ministr", "Předsed", "Poslan", "Ministr"]

In [121]:
with open(path, 'r', encoding="utf-8") as file:
    data2017 = yaml.safe_load(file)

In [122]:
schuze_polit = []

for schuze_id in list(data2017.keys()):
    for speaker in list(data2017[schuze_id].keys()):
        print(schuze_id, ": ", speaker)
        schuze_polit += [schuze_id]
        # if(key_word in speaker):
        #     print(data2017[schuze_id][speaker])

10. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
12. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
13. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
14. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
16. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
17. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
17. schůze :  Poslanec Miloš Babiš
19. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
19. schůze :  Poslanec Miloš Babiš
20. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
23. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
25. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
25. schůze :  Poslanec Miloš Babiš
26. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
27. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
28. schůze :  Místopředseda vlády ČR a ministr financí Andrej Babiš
29. schůze 

In [123]:
df_talks = pd.DataFrame({"speaker": list([""]), "new": list([""])})

for schuze_id in list(data2017.keys()):
    for speaker in list(data2017[schuze_id].keys()):
        for post in list(data2017[schuze_id][speaker]):
            first_word = post.split(" ")[0]
            new_speaker = False
            for imp_word in important_words:
                if(imp_word in first_word):
                    new_speaker = True
            df_talks = df_talks._append({"speaker": post, "new": new_speaker, "schuze": schuze_id}, ignore_index = True)

In [124]:
df_talks[df_talks.new == True]

,speaker,new,schuze
3,"Místopředseda PSP Jan Bartošek: Pane poslanče,...",True,10. schůze
5,"Poslanec Ludvík Hovorka: Dobře, děkuji.",True,10. schůze
7,Místopředseda PSP Jan Bartošek: Děkuji vám. Po...,True,10. schůze
9,Ministr zdravotnictví ČR Svatopluk Němeček Vá...,True,10. schůze
11,Místopředseda PSP Jan Bartošek: Děkuji vám. Ta...,True,10. schůze
...,...,...,...
1515,Místopředseda PSP Petr Gazdík: Děkuji. Takže n...,True,8. schůze
1517,Poslanec Miroslav Kalousek: Chtěl jsem se jeno...,True,8. schůze
1519,Místopředseda PSP Petr Gazdík: Děkuji panu pos...,True,8. schůze
1521,Poslanec Radek Vondráček: (Směje se.) Asi nena...,True,8. schůze


In [125]:
polit = []
prev_schuze = ""

for index, row in df_talks.iterrows():
    if(row.new):
        # splitted = row.speaker.split(":")
        if(key_word in row.speaker):
            polit += [True]
        else:
            polit += [False]
    else:
        if(polit == []):
            polit += [False]
        elif(prev_schuze != row.schuze):
            polit += [False]
        else:
            polit += [polit[-1]]
    prev_schuze = row.schuze

df_talks["polit"] = polit
print(df_talks)

                                                speaker    new      schuze  \
0                                                                      NaN   
1     \nKomu patří budovy na státním pozemku? Patří ...  False  10. schůze   
2                                                        False  10. schůze   
3     Místopředseda PSP Jan Bartošek: Pane poslanče,...   True  10. schůze   
4                                                        False  10. schůze   
...                                                 ...    ...         ...   
1523  Místopředseda PSP Petr Gazdík: Děkuji panu pos...   True   8. schůze   
1524                                                     False   8. schůze   
1525  Nejprve tedy budeme hlasovat o přikázání rozpo...  False   8. schůze   
1526  Zahajuji hlasování o přikázání tohoto návrhu z...  False   8. schůze   
1527  Je to hlasování číslo 170, přihlášeno 135 posl...  False   8. schůze   

      polit  
0     False  
1     False  
2     False  
3     F

In [126]:
df_sel = df_talks[df_talks.polit == True]
print(df_sel)

                                                speaker    new      schuze  \
30    Místopředseda vlády ČR a ministr financí Andre...   True  10. schůze   
31    Takže původně rozpočet předpokládal rozmrazení...  False  10. schůze   
32    Celkový objem prostředků příslušníků meziročně...  False  10. schůze   
33    Já rozumím, že tady došlo ke snížení o 10 %, a...  False  10. schůze   
34    Takže ve věci navýšení tarifních tabulek přísl...  False  10. schůze   
...                                                 ...    ...         ...   
1455  Místopředseda vlády ČR a ministr financí Andre...   True   7. schůze   
1456               (Jednání pokračovalo v 18.15 hodin.)  False   7. schůze   
1457                                                     False   7. schůze   
1503  Místopředseda vlády ČR a ministr financí Andre...   True   8. schůze   
1504                                                     False   8. schůze   

      polit  
30     True  
31     True  
32     True  
33     

In [127]:
polit_data = {}
prev_schuze = ""
speaker_intro = "Andrej Babiš: "

for schuze_id in schuze_polit:
    polit_data[schuze_id] = ""

for index, row in df_sel.iterrows():
    if(row.speaker != ""):
        if(speaker_intro in row.speaker):
            to_insert = row.speaker.split("Andrej Babiš: ")[1]
        else:
            to_insert = row.speaker

        polit_data[row.schuze] += to_insert.replace("\n", "")

print(polit_data)

{'10. schůze': 'Děkuji za slovo, pane předsedo. Dámy a pánové, já nevím, kdo komu co sliboval. Faktem je, že my jsme převzali tuto zemi v nějakém stavu, kdy minulá nebo předminulá vláda snížila ty platy, a samozřejmě děláme, co můžeme, protože my jsme se zavázali, že schodek nebude větší než 3 % HDP. A bohužel i ty instituce jako finanční úřady, celní úřady, úřady práce, které by měly vlastně zabezpečovat výnos daní, zabraňovat zlodějnám spotřební daně a odhalovat zneužívání sociálních dávek, jsou také poddimenzované, mají nízké platy. Takže původně rozpočet předpokládal rozmrazení, navýšení 1 %, dnes jsme na 3,5 % a samozřejmě jednání o rozpočtu budou pokračovat dál celé léto. Čekáme nový makroekonomický výhled v červenci. Rozumím tomu, co paní poslankyně říká. Celkový objem prostředků příslušníků meziročně vzroste zatím na 880 mil. Kč, to znamená, že to je nárůst o tisíc korun. V minulých letech platy příslušníků bezpečnostních sborů byly navyšovány už v roce 2014 o 2 %, dále bylo do

In [128]:
file_name = "./data/babis_preprocess2013.yaml"

with open(file_name, "w", encoding="utf-8") as file:
    yaml.dump(polit_data, file, allow_unicode=True)